In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder, LabelEncoder
import pickle

In [7]:
df=pd.read_csv("Churn_Modelling.csv")

In [8]:
df.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [9]:
df=df.drop(['RowNumber','CustomerId','Surname'],axis=1)

In [10]:
#encode categorical data
LabelEncoder_Gender=LabelEncoder()
df['Gender']=LabelEncoder_Gender.fit_transform(df['Gender'])
df.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,0,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,0,41,1,83807.86,1,0,1,112542.58,0
2,502,France,0,42,8,159660.80,3,1,0,113931.57,1
3,699,France,0,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,0,43,2,125510.82,1,1,1,79084.10,0


In [11]:
from sklearn.preprocessing import OneHotEncoder
OneHotEncoder_geo=OneHotEncoder()
geo_emco=OneHotEncoder_geo.fit_transform(df[['Geography']])
geo_emco

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 10000 stored elements and shape (10000, 3)>

In [12]:
OneHotEncoder_geo.get_feature_names_out(['Geography'])

array(['Geography_France', 'Geography_Germany', 'Geography_Spain'],
      dtype=object)

In [13]:
import pandas as pd
geo_df=pd.DataFrame(geo_emco.toarray(),columns=OneHotEncoder_geo.get_feature_names_out(['Geography']))

In [14]:
##combine one hot encoder columns with orignal data
data=pd.concat([df.drop('Geography',axis=1),geo_df],axis=1)

In [15]:
data.head()

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain
0,619,0,42,2,0.00,1,1,1,101348.88,1,1.0,0.0,0.0
1,608,0,41,1,83807.86,1,0,1,112542.58,0,0.0,0.0,1.0
2,502,0,42,8,159660.80,3,1,0,113931.57,1,1.0,0.0,0.0
3,699,0,39,1,0.00,2,0,0,93826.63,0,1.0,0.0,0.0
4,850,0,43,2,125510.82,1,1,1,79084.10,0,0.0,0.0,1.0


In [16]:
X=data.drop('EstimatedSalary',axis=1)
y=data['EstimatedSalary']

In [18]:
x_train,x_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=42)
scaler=StandardScaler()
x_train=scaler.fit_transform(x_train)
x_test=scaler.transform(x_test)

In [19]:
## save the encoders and sscaler
with open('label_encoder_gender.pkl','wb') as f:
    pickle.dump(LabelEncoder_Gender,f)
with open('onehot_encoder_geo.pkl','wb') as f:
    pickle.dump(OneHotEncoder_geo,f)
with open('scaler.pkl','wb') as f:
    pickle.dump(scaler,f)

    

## Ann regression

from t

In [20]:
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense

In [21]:
## Build Model
model=Sequential(
    [
        Dense(64,activation='relu',input_shape=(x_train.shape[1],)),
        Dense(32,activation='relu'),
        Dense(1)  #output layer for regression
    ]
)
model.compile(optimizer='adam',loss='mean_squared_error',metrics=['mae'])

C:\Users\a4ana\AppData\Roaming\Python\Python313\site-packages\keras\src\layers\core\dense.py:95: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [22]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 64)             │           832 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,945 (11.50 KB)

 Trainable params: 2,945 (11.50 KB)

 Non-trainable params: 0 (0.00 B)

In [23]:
from keras.callbacks import EarlyStopping
import datetime
log_dir="regression_logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback=tf.keras.callbacks.TensorBoard(log_dir=log_dir,histogram_freq=1)

In [24]:
EarlyStopping_monitor=EarlyStopping(monitor='val_loss',patience=10,mode='min',verbose=1)

In [25]:
## train the data
history=model.fit(
    x_train,y_train,
    validation_data=(x_test,y_test),
    epochs=100,
    callbacks=[tensorboard_callback,EarlyStopping_monitor])

Epoch 1/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 3s 4ms/step - loss: 13384829952.0000 - mae: 100383.0625 - val_loss: 13011569664.0000 - val_mae: 98541.6250
Epoch 2/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 13255385088.0000 - mae: 99734.7812 - val_loss: 12753172480.0000 - val_mae: 97238.4141
Epoch 3/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 12795498496.0000 - mae: 97457.1172 - val_loss: 12082534400.0000 - val_mae: 93857.6094
Epoch 4/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 11873203200.0000 - mae: 92888.1719 - val_loss: 10939361280.0000 - val_mae: 88075.2656
Epoch 5/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 10500974592.0000 - mae: 86022.8125 - val_loss: 9413261312.0000 - val_mae: 80394.3281
Epoch 6/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 8828170240.0000 - mae: 77666.7656 - val_loss: 7716255744.0000 - val_mae: 71933.2188
Epoch 7/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 7110743552.0000 - mae: 69040.6406 - val_loss: 6122685952.00

In [30]:
%load_ext tensorboard
!pip show tensorboard

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard
Name: tensorboard
Version: 2.20.0
Summary: TensorBoard lets you watch Tensors Flow
Home-page: https://github.com/tensorflow/tensorboard
Author: Google Inc.
Author-email: packages@tensorflow.org
License: Apache 2.0
Location: C:\Users\a4ana\AppData\Roaming\Python\Python313\site-packages
Requires: absl-py, grpcio, markdown, numpy, packaging, pillow, protobuf, setuptools, tensorboard-data-server, werkzeug
Required-by: tensorflow


In [32]:
import os
os.environ.pop('TENSORBOARD_BINARY', None)


'C:\\Users\\a4ana\\AppData\\Roaming\\Python\\Python313\\site-packages'

In [38]:

%tensorboard --logdir regression_logs/fit


ERROR: Could not find `tensorboard`. Please ensure that your PATH
contains an executable `tensorboard` program, or explicitly specify
the path to a TensorBoard binary by setting the `TENSORBOARD_BINARY`
environment variable.

In [39]:
test_loss,test_mae=model.evaluate(x_test,y_test)
print(f"Test MAE: {test_mae}")

63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 3348723200.0000 - mae: 50022.8320
Test MAE: 50022.83203125


In [41]:
model.save('salary_regression_model.h5')